In [ ]:
# 2025-4-26 run it on "python 3" base env on ins-gl-pt-gpu24-2c94136-3env-j4-l4-test-1 with 1 L4 GPU
# run on a single GPU notebook cell you can point the same script directly

In [ ]:
# 2025-5-4 confirm libcuda.so is available
# root@a085497e5485:~/mluser/git/llm-cv-pano-cubediff/cl/config# sudo find / -name "libcuda.so*" 2>/dev/null
# /usr/local/nvidia/lib64/libcuda.so
# /usr/local/nvidia/lib64/libcuda.so.1
# /usr/local/nvidia/lib64/libcuda.so.570.124.06
# /usr/local/cuda-12.4/targets/x86_64-linux/lib/stubs/libcuda.so
# /usr/local/cuda-12.4/compat/libcuda.so
# /usr/local/cuda-12.4/compat/libcuda.so.1
# /usr/local/cuda-12.4/compat/libcuda.so.550.127.08
# root@a085497e5485:~/mluser/git/llm-cv-pano-cubediff/cl/config# ldconfig -p | grep libcuda
#         libcudart.so.12 (libc6,x86-64) => /usr/local/cuda/targets/x86_64-linux/lib/libcudart.so.12
#         libcudart.so (libc6,x86-64) => /usr/local/cuda/targets/x86_64-linux/lib/libcudart.so

In [1]:
!pip install torch==2.1.2 torchvision==0.16.2 "peft>=0.8.2" \
     opencv-python==4.8.1.78 opencv-contrib-python==4.8.1.78 matplotlib==3.8.2 tqdm==4.66.1 einops==0.7.0 \
     opencv-python xformers requests pillow openexr opencv-python-headless wandb \
     bitsandbytes webdataset timm diffusers huggingface_hub accelerate

!pip uninstall torch-xla -y

  Using cached torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached torchvision-0.16.2-cp310-cp310-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached peft-0.15.2-py3-none-any.whl.metadata (13 kB)
  Using cached opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached opencv_contrib_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached matplotlib-3.8.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached einops-0.7.0-py3-none-any.whl.metadata (13 kB)
  Using cached xformers-0.0.30-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (1.0 kB)
  Using cached openexr-3.3.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 k

In [6]:
# !pip install --upgrade accelerate
# !pip install --upgrade diffusers>=0.34.0
!pip install --upgrade diffusers xformers

  Using cached xformers-0.0.30-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (1.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 96.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 51.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 55.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 42.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 91.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 40.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 60.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 68.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# force pip to skip its cache and grab the latest main branch
!pip install --upgrade --no-cache-dir \
    git+https://github.com/huggingface/peft.git \
    git+https://github.com/huggingface/diffusers.git


In [3]:
import torch, os; print(os.path.join(torch.__path__[0],'lib'))

/opt/conda/lib/python3.10/site-packages/torch/lib


In [1]:
!pip list | grep "dif\|hug\|trans\|torch\|acce\|peft\|bits\|xf"

accelerate                               1.6.0
bitsandbytes                             0.45.5
diffusers                                0.33.1
huggingface-hub                          0.30.2
peft                                     0.15.2
torch                                    2.1.2
torchvision                              0.16.2
transformers                             4.51.3
xformers                                 0.0.23.post1


In [1]:
# 4. Configure xFormers if available
import torch, os

if hasattr(torch, "_dynamo"):
    torch._dynamo.config.suppress_errors = True
    # Disable dynamo to avoid cudnnGetLibConfig errors
    import torch._dynamo
    torch._dynamo.reset()
    torch._dynamo.disable()
    print("✅ Disabled torch._dynamo")
    
try:
    import xformers
    # Ensure PyTorch uses high precision for matmul operations
    if hasattr(torch, "set_float32_matmul_precision"):
        torch.set_float32_matmul_precision('high')
    
    # Force xFormers to use bfloat16 for attention operations
    os.environ["XFORMERS_FORCE_MIXED_PREC"] = "bfloat16"
    print("✅ Configured xFormers for bfloat16 precision")
    # return True
except ImportError:
    print("⚠️ xFormers not available")

✅ Disabled torch._dynamo
✅ Configured xFormers for bfloat16 precision


In [ ]:
import os
os.environ["PYTHONPATH"] = "/home/jupyter/mluser/git/llm-cv-pano-cubediff"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
# Set torch compile backend
os.environ["TORCH_COMPILE_BACKEND"] = "inductor"
os.environ["ACCELERATE_CONFIG_FILE"]="/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/config/accelerate_config.yaml"
# export LD_LIBRARY_PATH="$(python -c "import torch, os; print(os.path.join(torch.__path__[0],'lib'))"):$LD_LIBRARY_PATH"
# os.environ["LD_LIBRARY_PATH"]="/opt/conda/lib/python3.10/site-packages/torch/lib:" + os.environ.get("LD_LIBRARY_PATH", "")
# os.environ["TORCH_LOGS"] = "+dynamo"
# os.environ["TORCHDYNAMO_VERBOSE"] = "1"

# TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1

# 2025-5-4 run this from a terminal where the code repo will be run to load libcuda.so beforehand
# !ldconfig /usr/local/nvidia/lib64

In [2]:
# print(f"accelerate/utils/operations.py - line 612 concatenate() - data type is {type(data)}, data[0] type is {type(data[0])}")
# if isinstance(data[0], str):
#    print(f"data[0] type is {type(data[0])}, it is {data[0]}")
# elif isinstance(data[0], dict):
#    print(f"data[0] type is {type(data[0])}, keys is {[k for k in data[0].keys()]}")
# elif isinstance(data[0], list):
#    print(f"data[0] type is {type(data[0])} size is {len(data[0])}")
#    for x in data[0]:
#        if isinstance(x, dict):
#            print(f"x type is {type(x)}, size is {len(x)}, x keys is {[k for k in x.keys()]}")
#        elif isinstance(x, torch.Tensor):
#            print(f"x type is {type(x)}, shape is {x.shape}")
#        else:
#            print(f"x type is {type(x)}, it is {x}")
# elif isinstance(data[0], torch.Tensor):
#    print(f"data[0] type is {type(data[0])}, shape is {data[0].shape}")
# else:
#    print(f"data[0] type is {type(data[0])}") 

In [ ]:
# !accelerate launch \
#   --mixed_precision=bf16 \
#   --num_processes=4 \
#   ./train_cubediff.py --cfg tiny_lora.yaml

# transparently CPU-offload part of the model or optimizer state
# that lets you push some weights or optimizer buffers to host memory.
# !accelerate launch \
#     --mixed_precision=bp16 \
#     --num_processes=4 \
#     --offload_folder=/tmp/ofa offload_optimizer=True offload_parameters=True \
#     train_cubediff.py --cfg tiny_lora.yaml

In [ ]:
# !accelerate config --config_file ../config/accelerate_config.yaml

--------------------------------------------------------------------------------In which compute environment are you running?
Please select a choice using the arrow or number keys, and selecting with enter
 ➔  This machine
    AWS (Amazon SageMaker)
Traceback (most recent call last):
  File "/opt/conda/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.10/site-packages/accelerate/commands/accelerate_cli.py", line 50, in main
    args.func(args)
  File "/opt/conda/lib/python3.10/site-packages/accelerate/commands/config/config.py", line 67, in config_command
    config = get_user_input()
  File "/opt/conda/lib/python3.10/site-packages/accelerate/commands/config/config.py", line 32, in get_user_input
    compute_environment = _ask_options(
  File "/opt/conda/lib/python3.10/site-packages/accelerate/commands/config/config_utils.py", line 63, in _ask_options
    result = menu.run(default_choice=default)
  File "/opt/conda/lib/python3.10/site-packages/acc

In [ ]:
# allocate more memory for shm
# !sudo mount -o remount,size=8G /dev/shm

In [ ]:
ACCELERATE_CONFIG_FILE="/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/config/accelerate_config.yaml"

In [ ]:
# !python ./train_cubediff.py --cfg ../config/tiny_lora.yaml
# !accelerate launch train_cubediff.py --cfg tiny_lora.yaml
!accelerate launch --config_file $ACCELERATE_CONFIG_FILE train_cubediff.py --cfg ../config/tiny_lora.yaml > cubediff_log.txt

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
^C
Traceback (most recent call last):
  File "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/training/train_cubediff.py", line 9, in <module>
Traceback (most recent call last):
  File "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/training/train_cubediff.py", line 9, in <module>
    import fix_attention_dtype
  File "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/training/fix_attention_dtype.py", line 8, in <module>
    import fix_attention_dtype
  File "/home/jupyter/mluser/git/llm-c

In [ ]:
# 2025-5-6 error of dtypemismatch for q, k, v
# diffusers/models/attention_processor.py - 3106 line - xformers.ops.memory_efficient_attention called with - query dtype is torch.float32, key dtype is torch.bfloat16, value dtype is torch.bfloat16
# Traceback (most recent call last):
#   File "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/training/./train_cubediff.py", line 97, in <module>
#     main()
#   File "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/training/./train_cubediff.py", line 94, in main
#     trainer.train()                       # ← generates samples & checkpoints
#   File "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/training/trainer.py", line 396, in train
#     pred = self.model(
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
#     return self._call_impl(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1534, in _call_impl
#     return forward_call(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/accelerate/utils/operations.py", line 814, in forward
#     return model_forward(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/accelerate/utils/operations.py", line 802, in __call__
#     return convert_to_fp32(self.model_forward(*args, **kwargs))
#   File "/opt/conda/lib/python3.10/site-packages/torch/amp/autocast_mode.py", line 16, in decorate_autocast
#     return func(*args, **kwargs)
#   File "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/model/architecture.py", line 215, in forward
#     unet_out = self.base_unet(
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
#     return self._call_impl(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1534, in _call_impl
#     return forward_call(*args, **kwargs)
#   File "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/training/peft_patch.py", line 34, in filtered_forward
#     return original_forward(self, *args, **filtered_kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/peft/peft_model.py", line 818, in forward
#     return self.get_base_model()(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
#     return self._call_impl(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1534, in _call_impl
#     return forward_call(*args, **kwargs)
#   File "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/training/peft_patch.py", line 34, in filtered_forward
#     return original_forward(self, *args, **filtered_kwargs)
#   File "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/training/trainer.py", line 70, in _patched_unet_forward
#     return orig_unet_forward(self, sample, timestep, encoder_hidden_states)
#   File "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/training/fix_pytorch_issues.py", line 60, in simple_forward
#     return original_forward(self, sample, timestep, encoder_hidden_states)
#   File "/opt/conda/lib/python3.10/site-packages/diffusers/models/unets/unet_2d_condition.py", line 1216, in forward
#     sample, res_samples = downsample_block(
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
#     return self._call_impl(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1534, in _call_impl
#     return forward_call(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/diffusers/models/unets/unet_2d_blocks.py", line 1260, in forward
#     hidden_states = attn(
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
#     return self._call_impl(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1534, in _call_impl
#     return forward_call(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py", line 416, in forward
#     hidden_states = self._gradient_checkpointing_func(
#   File "/opt/conda/lib/python3.10/site-packages/diffusers/models/modeling_utils.py", line 320, in _gradient_checkpointing_func
#     return torch.utils.checkpoint.checkpoint(
#   File "/opt/conda/lib/python3.10/site-packages/torch/_compile.py", line 24, in inner
#     return torch._dynamo.disable(fn, recursive)(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py", line 328, in _fn
#     return fn(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/_dynamo/external_utils.py", line 17, in inner
#     return fn(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py", line 458, in checkpoint
#     ret = function(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
#     return self._call_impl(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1534, in _call_impl
#     return forward_call(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/diffusers/models/attention.py", line 556, in forward
#     attn_output = self.attn2(
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
#     return self._call_impl(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1534, in _call_impl
#     return forward_call(*args, **kwargs)
#   File "/opt/conda/lib/python3.10/site-packages/diffusers/models/attention_processor.py", line 605, in forward
#     return self.processor(
#   File "/opt/conda/lib/python3.10/site-packages/diffusers/models/attention_processor.py", line 3107, in __call__
#     hidden_states = xformers.ops.memory_efficient_attention(
#   File "/opt/conda/lib/python3.10/site-packages/xformers/ops/fmha/__init__.py", line 223, in memory_efficient_attention
#     return _memory_efficient_attention(
#   File "/opt/conda/lib/python3.10/site-packages/xformers/ops/fmha/__init__.py", line 326, in _memory_efficient_attention
#     return _fMHA.apply(
#   File "/opt/conda/lib/python3.10/site-packages/torch/autograd/function.py", line 539, in apply
#     return super().apply(*args, **kwargs)  # type: ignore[misc]
#   File "/opt/conda/lib/python3.10/site-packages/xformers/ops/fmha/__init__.py", line 42, in forward
#     out, op_ctx = _memory_efficient_attention_forward_requires_grad(
#   File "/opt/conda/lib/python3.10/site-packages/xformers/ops/fmha/__init__.py", line 348, in _memory_efficient_attention_forward_requires_grad
#     inp.validate_inputs()
#   File "/opt/conda/lib/python3.10/site-packages/xformers/ops/fmha/common.py", line 121, in validate_inputs
#     raise ValueError(
# ValueError: Query/Key/Value should either all have the same dtype, or (in the quantized case) Key/Value should have dtype torch.int32
#   query.dtype: torch.float32
#   key.dtype  : torch.bfloat16
#   value.dtype: torch.bfloat16

# # File "/opt/conda/lib/python3.10/site-packages/diffusers/models/attention.py", line 556, in forward


In [3]:
!pip install torch==2.1.2

  Using cached torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl.m

In [4]:
!pip install xformers==0.0.23.post1

  Using cached xformers-0.0.23.post1-cp310-cp310-manylinux2014_x86_64.whl.metadata (1.0 kB)
Using cached xformers-0.0.23.post1-cp310-cp310-manylinux2014_x86_64.whl (213.0 MB)
  Attempting uninstall: xformers
    Found existing installation: xformers 0.0.30
    Uninstalling xformers-0.0.30:
      Successfully uninstalled xformers-0.0.30


In [3]:
!pip list | grep "dif\|hug\|trans\|torch\|acce\|peft\|bits\|xf"

# 2025-5-6 due to xformers, it got :
# ValueError: Query/Key/Value should either all have the same dtype, or (in the quantized case) Key/Value should have dtype torch.int32
#   query.dtype: torch.float32
#   key.dtype  : torch.bfloat16
#   value.dtype: torch.bfloat16

# accelerate                               1.6.0
# bitsandbytes                             0.45.5
# diffusers                                0.33.1
# huggingface-hub                          0.30.2
# peft                                     0.15.2
# torch                                    2.1.2
# torchvision                              0.16.2
# transformers                             4.51.3
# xformers                                 0.0.23.post1


# accelerate                               1.6.0
# bitsandbytes                             0.45.5
# diffusers                                0.33.1
# huggingface-hub                          0.30.2
# peft                                     0.15.2
# torch                                    2.7.0
# torchvision                              0.16.2
# transformers                             4.51.3
# xformers                                 0.0.30

accelerate                               1.6.0
bitsandbytes                             0.45.5
diffusers                                0.33.1
huggingface-hub                          0.30.2
peft                                     0.15.2
torch                                    2.1.2
torchvision                              0.16.2
transformers                             4.51.3
xformers                                 0.0.23.post1


In [3]:
import torch
import xformers

print(torch.__version__)     # should be 2.1.2+cu121 or similar
# 2.1.2 + cu121

print(xformers.__version__)  # should be 0.0.23.post1


2.1.2+cu121
0.0.23.post1


In [4]:
!nvidia-smi

Tue May  6 17:51:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   42C    P8             16W /   72W |       1MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch, os; print(os.path.join(torch.__path__[0],'lib'))

/opt/conda/lib/python3.10/site-packages/torch/lib


In [5]:
# !ls -lh /usr/local/nvidia/lib64/libcuda.so

lrwxrwxrwx 1 root root 12 Apr 15 07:15 /usr/local/nvidia/lib64/libcuda.so -> libcuda.so.1
